In [1]:
from WNQuery import *

In [ ]:
warnings_file = open('warnings.txt','w')
wnquery = WNQuery("plwordnet_3_1/plwordnet-3.1-visdisc.xml", log=warnings_file)

In [ ]:
import sys
wnquery.writeStats(sys.stdout)

In [ ]:
# Znajdź wszystkie znaczenia rzeczownika szkoda oraz wymień ich synonimy (jeśli posiadają).
meanings = wnquery.lookUpLiteral('szkoda', 'n')
for meaning in meanings:
    print(meaning.toString())

In [ ]:
# Znajdź domknięcie przechodnie relacji hiperonimi dla pierwszego znaczenia wyrażenia wypadek drogowy 
# i przedstaw je w postaci grafu skierowanego.
meaning = wnquery.lookUpSense('wypadek drogowy', 1, 'n')
graph = []

def traceRelation(wnquery, wnid, pos, rel):
    res = []
    ids = wnquery.lookUpRelation(wnid, pos, rel)
    res.append(wnid)
    print(wnquery.getSynset(wnid, 'n').toString())
    for i in ids:  
        graph.append((wnid, i))
        res.extend(traceRelation(wnquery, i, pos, rel))
    return res

traceRelation(wnquery, meaning.wnid, 'n', 'hypernym')

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

G = nx.DiGraph()
G.add_edges_from(graph)
plt.figure(figsize=(12,3)) 
nx.draw_spring(G, with_labels = True)
plt.show()

In [ ]:
# Znajdź bezpośrednie hiponimy rzeczownika wypadek1.
meaning = wnquery.lookUpSense('wypadek', 1, 'n')
relation = wnquery.lookUpRelation(meaning.wnid, meaning.pos, 'hyponym')
for meaning in relation:
    print(wnquery.getSynset(meaning, 'n').toString())

In [ ]:
# Znajdź hiponimy drugiego rzędu dla rzeczownika wypadek1.
meaning = wnquery.lookUpSense('wypadek', 1, 'n')
relation = wnquery.lookUpRelation(meaning.wnid, meaning.pos, 'hyponym')
for meaning in relation:
    synset = wnquery.getSynset(meaning, 'n')
    relations = wnquery.lookUpRelation(synset.wnid, synset.pos, 'hyponym')
    for rel in relations:
        print(wnquery.getSynset(rel, 'n').toString())

In [ ]:
# Przedstaw w postaci grafu skierowanego (z etykietami dla krawędzi) relacje semantyczne pomiędzy następującymi grupami leksemów:
# szkoda2, strata1, uszczerbek1, szkoda majątkowa1, uszczerbek na zdrowiu1, krzywda1, niesprawiedliwość1, nieszczęście2.
literals1 = {'szkoda': 1, 'strata': 1, 'uszczerbek': 1, 'szkoda majątkowa': 1, 'uszczerbek na zdrowiu': 1, 'krzywda': 1,
             'niesprawiedliwość': 1, 'nieszczęście': 2}

# wypadek1, wypadek komunikacyjny1, kolizja2, zderzenie2, kolizja drogowa1, bezkolizyjny2, 
# katastrofa budowlana1, wypadek drogowy1.
literals2 = {'wypadek': 1, 'wypadek komunikacyjny': 1, 'kolizja': 2, 'zderzenie': 2, 'kolizja drogowa': 1, 
             'katastrofa budowlana': 1, 'wypadek drogowy': 1}

def get_synset_literals(synset, literals):
    return '\n'.join(filter(lambda x: x in literals, [synonym.literal for synonym in synset.synonyms]))

import itertools
def draw_relations(literals, skip=[]):
    synsets = [wnquery.lookUpSense(literal, pos, 'n') for literal, pos in literals.items()]
    synsets_ids = [synset.wnid for synset in synsets]
    
    G = nx.DiGraph()
    meaningful_relations = {}
    for synset in synsets:
        for target_id, relation_type in synset.ilrs:
            if target_id in synsets_ids:
                if relation_type not in skip:
                    G.add_edge(get_synset_literals(synset, literals), 
                               get_synset_literals(wnquery.lookUpID(target_id, 'n'), literals), 
                               rel=relation_type)

    plt.figure(figsize=(20,20)) 
    G.add_edges_from(meaningful_relations.keys())
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, arrows=False)
    nx.draw_networkx_edge_labels(G, pos, label_pos=0.2)
    plt.show()
    
#prevent overlapping of labels
draw_relations(literals1)
draw_relations(literals2)